In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from sklearn.model_selection import train_test_split
import random
import numpy as np
from tqdm import tqdm
import argparse
import wandb
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs
import rasterio
from model import *
from dataset import *
from utils import *
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
class SixImageDataset_DEM_GT_Geo(Dataset):
    def __init__(self, pos_dir, 
                 neg_dir, 
                 pos_dem_dir, 
                 neg_dem_dir, 
                 pos_gt_mask_dir, 
                 neg_gt_mask_dir, 
                 transform=None):
        self.data = []
        self.labels = []
        self.dem_paths = []
        self.gt_mask_paths = []
        self.geo_info = []

        # Load all files and group by tiles
        pos_files = [f for f in os.listdir(pos_dir) if f.endswith('.tif')]
        neg_files = [f for f in os.listdir(neg_dir) if f.endswith('.tif')]
        pos_tiles = self.group_files_by_tile(pos_files)
        neg_tiles = self.group_files_by_tile(neg_files)

        # Handle class imbalance by oversampling the minority class
        max_len = max(len(pos_tiles), len(neg_tiles))
        if len(pos_tiles) > len(neg_tiles):
            neg_tiles = self.oversample(neg_tiles, max_len)
        else:
            pos_tiles = self.oversample(pos_tiles, max_len)

        # Combine and store
        self.store_tiles(pos_tiles, pos_dir, pos_dem_dir, pos_gt_mask_dir, 1)
        self.store_tiles(neg_tiles, neg_dir, neg_dem_dir, neg_gt_mask_dir, 0)
        
        self.transform = transform

    def group_files_by_tile(self, files):
        tile_dict = {}
        for file in files:
            tile_number = file.split('_')[-1].split('.')[0]
            if tile_number not in tile_dict:
                tile_dict[tile_number] = []
            tile_dict[tile_number].append(file)
        # Only include complete groups
        return [tile for tile in tile_dict.values() if len(tile) == 6]

    def oversample(self, tiles, target_length):
        # Repeat tiles until the desired length is achieved
        return random.choices(tiles, k=target_length)

    def store_tiles(self, tiles, directory, dem_dir, gt_mask_dir, label):
        for tile_files in tiles:
            tile_number = tile_files[0].split('_')[-1].split('.')[0]
            self.data.append([os.path.join(directory, f) for f in sorted(tile_files)])
            self.dem_paths.append(os.path.join(dem_dir, f"dem_tile_{tile_number}.tif"))
            if label == 1:
                self.gt_mask_paths.append(os.path.join(gt_mask_dir, f"ground_truth_tile_{tile_number}.tif"))
            else: 
                self.gt_mask_paths.append(os.path.join(gt_mask_dir, f"negative_ground_truth_tile_{tile_number}.tif"))
            self.labels.append(label)
            self.geo_info.append(self.extract_geo_info(os.path.join(directory, tile_files[0])))

    def extract_geo_info(self, file_path):
        with rasterio.open(file_path) as dataset:
            geo_transform = dataset.transform
            crs = dataset.crs
        return {'geo_transform': geo_transform, 'crs': crs}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        images_1 = [imageio.imread(img_path).astype('uint8') for img_path in self.data[idx]]
        images = [transforms.functional.to_pil_image(image) for image in images_1]
        dem_image = transforms.functional.to_pil_image(imageio.imread(self.dem_paths[idx]).astype('uint8'))
        gt_mask = transforms.functional.to_pil_image(imageio.imread(self.gt_mask_paths[idx]).astype('uint8'))
        if self.transform:
            images = [self.transform(image) for image in images]
            dem_image = self.transform(dem_image)
            gt_mask = self.transform(gt_mask)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        geo_info = self.geo_info[idx]
        return images, dem_image, gt_mask, label, geo_info

In [4]:
pos_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_pos/rgb_images/'
neg_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_neg/rgb_images/'

pos_dem_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_pos/dem/'
neg_dem_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_neg/dem/'

pos_gt_mask_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_pos/ground_truth/'
neg_gt_mask_dir = '/home/macula/SMATousi/Gullies/ground_truth/organized_data/All_Pos_Neg/all_neg/ground_truth/'


In [7]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

full_dataset = SixImageDataset_DEM_GT_Geo(pos_dir, 
                             neg_dir, 
                             pos_dem_dir,
                             neg_dem_dir,
                             pos_gt_mask_dir,
                             neg_gt_mask_dir,
                             transform=transform)

In [9]:
arg_batch_size = 1
learning_rate = 0.0001
epochs = 1
number_of_workers = 0
# image_size = arg_imagesize
val_percent = 0.1

In [10]:
n_val = int(len(full_dataset) * val_percent)
n_train = len(full_dataset) - n_val
train, val = random_split(full_dataset, [n_train, n_val])

train_loader = DataLoader(train, batch_size=arg_batch_size, shuffle=True, num_workers=number_of_workers, pin_memory=True)
val_loader = DataLoader(val, batch_size=arg_batch_size, shuffle=False, num_workers=number_of_workers, pin_memory=True, drop_last=True)
    

In [11]:
model = Gully_Classifier(input_size=6*2048, hidden_size=512, output_size=1)

/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
